<img src="images/intro-logo-scala-eng.png" align="left" width="600px"/>
<!--- ![alt text](heading.png "Heading with Scala logo") --->

---

# Index

### [3. Now, let's run!](#seccion-3. Now, lets run!)
  * [More about collections](#subseccion-More about collections)
  * [More about functions](#subseccion-More about functions)
    * [Internal functions](#subsubseccion-Internal functions)
    * [Function literals and functions as values](#subsubseccion-Function literals and functions as values)
    * [Partially applied functions](#subsubseccion-Partially applied functions)
    * [Closures](#subsubseccion-Closures)
    * [Variable parameters](#subsubseccion-Variable parameters)
    * [Higher-order functions](#subsubseccion-Higher-order functions)
  * [Currying](#subseccion-Currying)
  * [Parameters 'by-name' and 'lazy' vals](#subseccion-Parameters by-name and lazy vals)
  * [Tail recursion](#subseccion-Tail recursion)
  * [Companion `object`](#subseccion-Companion `object`)
  * [Covariant and contravariant types](#subseccion-Covariant and contravariant types)
  * [Implicit conversions](#subseccion-Implicit conversions)
  * [Pattern matching and `case` classes](#subseccion-Pattern matching and `case` classes)
  * [Type `Option`](#subseccion-Type `Option`)
  * [Extractors](#subseccion-Extractors)
    * [Regular expressions ](#subsubseccion-Regular expressions )


---

<a name="seccion-3. Now, lets run!"></a>
<a name="seccion-3. Ahora, ¡a correr!"></a>
<table align="left" style="border-collapse: collapse; width: 100%; border: 5px double black">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 100px;">
<img src="icons/correr-m.png" align="left" width=90px/>
        </td>
        <td style="border:none !important; text-align:left;">
<h1>3. Now, let's run!</h1>
<br>
It's time to learn some more advanced features of Scala, that will allow us to make the most of the language.
        </td>
    </tr>
</table>

<a name="subseccion-Más sobre colecciones"></a>
<a name="subseccion-More about collections"></a>
## More about collections


Having studied some of the most popular collections, let's have a look to other types intended to store data and apply operators on them.

* `ListBuffer`: as we already previously stated, perform operations on the last elements of a list is not a good idea, so, instead of using `reverse` to avoid the problem, we also could use a `ListBuffer`, that allows us to work efficiently on the first and last elements of a list  with its operators `+:` and `+=:`. Afterwards, we can transform it to a `List` with `toList`.
* `ArrayBuffer`: like `Array` but we do not need to specify the size because we can add new elements with `+=`.
* `Queue`: to define FIFO structures, there are `mutable` and `immutable` versions. With the first one, we use `+=` and `++=` in order to insert new elements and `dequeue` to extract them. In the `immutable` version, we use `enqueue` and `dequeue` to insert and extract elements.
* `Stack`: to define LIFO structures, also with `mutable` and `immutable` versions. Using `push`, `pop` and `top` to manipulate them.
* `TreeSet` and `TreeMap`: versions of `Set` and `Map` implementing `SortedSet` and `SortedMap` traits, respectively, being able to mantain the keys and values ordered.

In [ ]:
import scala.collection.mutable.ListBuffer

val listBuf = new ListBuffer[Int]
listBuf += 1
3 +=: listBuf
listBuf.toList

import scala.collection.mutable.ArrayBuffer

val arrBuf = new ArrayBuffer[Int]()
arrBuf += 1
arrBuf += 2
arrBuf(1)

import scala.collection.mutable.Queue

val cola = Queue[Int]()
cola += 1
cola += 2
cola ++= List(3, 4)
val elemento = cola.dequeue

import scala.collection.immutable.{Queue => ImQueue}

val colaVacia = ImQueue[Int]()
val cola1 = colaVacia.enqueue(1)
val (elemento2, colaVacia2) = cola1.dequeue

import scala.collection.mutable.Stack

val pila = Stack[Int]()
pila.push(1)
pila.push(2)
val elemento3 = pila.top
val elemento4 = pila.pop

import scala.collection.immutable.TreeSet

val miSetOrdenado = TreeSet(5, 2, 7, 9, 1)

<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/notepad.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important; text-align:left">
            <ul>
                <li>If we use `+:` instead of `+=:` a new (unmodified) instance of the original `ListBuffer` will be returned</li>
                <li>The strings can also be seen as sequences thanks to their implicit conversion to to `RichString`, that we will see later</li>
            </ul>
        </td>
    </tr>
</table>

<a name="subseccion-More about functions"></a>
## More about functions


Let us review some concepts about functions that can be different from other languages.

<a name="subsubseccion-Internal functions"></a>
### Internal functions

If we need to define a local function, hidden from outside the object, we can use `private` as seen previously, but in Scala we can achieve the same effect by defining the function to be hidden inside the function that uses it (i.e. we can define a function `processLine` inside function `processFile`). 

In [1]:
import scala.io.Source

object LongLines {
    def processFile(filename: String, width: Int) {
        def processLine(line: String) {     //Function defined inside other function (hidden for the rest of the code)
            if (line.length > width)
                print(filename +": "+ line)
        }
        val source = Source.fromFile(filename)
        for (line <- source.getLines)
            processLine(line)
    }
}

LongLines.processFile("files/longitud.txt", 45)

Intitializing Scala interpreter ...

Spark Web UI available at http://172.17.0.2:4040
SparkContext available as 'sc' (version = 2.2.0, master = local[*], app id = local-1515434138216)
SparkSession available as 'spark'


files/longitud.txt: Segunda línea con una longitud bastante mayor que el resto (63)

import scala.io.Source
defined object LongLines


<a name="subsubseccion-Function literals and functions as values"></a>
### Function literals and functions as values

Functions can be used without a name, just by defining them directly where they are required. That function literal will be compiled in a class in running time and it will be instantiated in an object used as a value.

Each function literal used as a value is transformed in a instance of a class extending the trait `FunctionN`, being `N` the number of params of the function. Scala has the definitions for traits from `Function0` to `Funtion22`, each of them implementing method `apply` that invokes the function.

Function literal can contain more than one sentence between braces and the returned values can be used in many places, such as parameters of functions like `foreach` or `filter` (seen previously) to specify the function to be applied to the elements of a sequence or the criteria to filter them.

In [1]:
(x: Int) => x + 1                   // Function literal
var incr = (x: Int) => x + 1        // Function literal assigned to a variable (with type Int => Int)
incr(2)                             // Calling the function
incr = (x: Int) => x + 2            // Reassigning the variable to other function literal (with type (Int, Int) => Int)
incr(2)                             // Calling the function
//incr = (x: Int, n: Int) => x + n  // What will happnen if the variable is reassigned to another type of function? (i.e. (Int, Int) => Int)

val nums = List(-1,0,1,2,3)
nums.foreach((x: Int) => print(x+" "))  // Function literal as a parameter

println()
nums.foreach((x: Int) => {              // Function literal with sevral lines
     val valido = x > 0                 // in this case we need to use the braces
     if (valido)
         for (y <- 0 to x)
            print("[X="+x+",Y="+y+"] ")
    })

println()
nums.filter((x: Int) => x > 0).        // calling filter and foreach to obtain the same result
    foreach((x: Int) =>
         for (y <- 0 to x)
            print("[X="+x+",Y="+y+"] ")
    )

Intitializing Scala interpreter ...

Spark Web UI available at http://172.17.0.2:4040
SparkContext available as 'sc' (version = 2.2.0, master = local[*], app id = local-1515484827858)
SparkSession available as 'spark'


-1 0 1 2 3 
[X=1,Y=0] [X=1,Y=1] [X=2,Y=0] [X=2,Y=1] [X=2,Y=2] [X=3,Y=0] [X=3,Y=1] [X=3,Y=2] [X=3,Y=3] 
[X=1,Y=0] [X=1,Y=1] [X=2,Y=0] [X=2,Y=1] [X=2,Y=2] [X=3,Y=0] [X=3,Y=1] [X=3,Y=2] [X=3,Y=3] 

incr: Int => Int = <function1>
incr: Int => Int = <function1>
nums: List[Int] = List(-1, 0, 1, 2, 3)


<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/notepad.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important;text-align:left;">
            <ul>
                <li>The fact that Scala defines the traits from `Function0` to `Function22` is the reason why functions in Scala can have 22 parameters or less</li>
            </ul>
        </td>
    </tr>
</table>

<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/optimizar.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important;text-align:left;">
            <ul>
                <li>In most cases we could avoid the type of the function literals because they can be inferred by the compiler. In those cases we could also avoid the brackets (`nums.filter(x => x > 0)`)</li>
                <li>We can reduce the code a bit more by using `_` to make reference to the parameters, only if they appear once in the body of the function, corresponding the first `_` to the first parameter, and so. In this way, we could avoid the left part of the function literal (`nums.filter(_ > 0)`)</li>
                <li>When using `_` we could be required to specify the type, which can be done between brackets (`val f = (_: Int) + (_: Int)`)</li>
            </ul>
        </td>
    </tr>
</table>

<a name="subsubseccion-Partially applied functions"></a>
### Partially applied functions

Here we show another utility of `_`, in this case to substitute the parameters in a given function. This mechanism allows us to wrap a function in a function literal that can be passed as a parameter or assigned to a variable (for example, try to assign function `suma3` to variable `fs3` without `_`).

On the other hand, instead of substitute all the parameters with `_`, we could also substitute just a couple of them. In this case we are creating a wrapping function with less parameters than the original one because we just need to provide the parameters substituted by the `_`.

In [2]:
def suma3(a: Int, b: Int, c: Int) = a + b + c
val fs3 = suma3 _   // Type of 'suma3 _' is a generated class with an `apply` method of 3 parameters
fs3(1, 2, 3)        // Calling the function
fs3.apply(1, 2, 3)  // This is equivalent

val fs3p = suma3 (1, (_: Int), 3)
fs3p(2)
fs3p.apply(2)       // This instruction will call suma3(1, 2, 3)

suma3: (a: Int, b: Int, c: Int)Int
fs3: (Int, Int, Int) => Int = <function3>
fs3p: Int => Int = <function1>
res1: Int = 6


<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/optimizar.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important;text-align:left;">
            <ul>
                <li>We can avoid the use of `_` when substituting all the parameters of a function anywhere in the code where a function is expected (`nums.foreach(println)`)</li>
            </ul>
        </td>
    </tr>
</table>

<a name="subsubseccion-Closures"></a>
### Closures

In a function literal like `(x: Int) => x + 1` all the necessary elements to compute its value are present. It can happen that the function needs to access to external variables, like in `(x: Int) => x + count`, in these cases it is said that the function has a "free" variable and the function value is called a **closure**. This is a very useful mechanism, for example to define a function inside other function in such way that the internal function could have access to the variables of the container function.

In Scala, if the value of the "free" variable changes, the closure will be aware of that, as you can see in the example below.

Finally, if the closure have access to a free variable with many copies it will refer to the one that was active when the closure was created.

In [4]:
var cantidad = 1
val sumaCantidad = (x: Int) => x + cantidad
sumaCantidad(1)
cantidad = 10
sumaCantidad(1)

def makeIncreaser(cantidad: Int) = (x: Int) => x + cantidad // Creates a new closurein each call
val inc1 = makeIncreaser(1)
val inc10 = makeIncreaser(10)
inc1(1)
inc10(1)


cantidad: Int = 10
sumaCantidad: Int => Int = <function1>
cantidad: Int = 10
makeIncreaser: (cantidad: Int)Int => Int
inc1: Int => Int = <function1>
inc10: Int => Int = <function1>
res3: Int = 11


<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/notepad.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important;text-align:left;">
            <ul>
                <li>A closure can also change the value of a free variable</li>
            </ul>
        </td>
    </tr>
</table>

<a name="subsubseccion-Variable parameters"></a>
### Variable parameters

In Scala we can define a function with a variable number of parameters of the same type by using `*` at the end of the type. 

Nevertheless, if we have an `Array` of the appropriate type, T, we cannot pass it directly as a parameter of the mentioned function, but we must add `: _*` after the array, as shown in the example. Otherwise, we will be passing just one parameter of type `Array[T]`, instead of a number of parameters of type T. 

In [5]:
def echo(args: String*) = 
    for (arg <- args) println(arg)

echo()
echo("Hola")
echo("Hola", "mundo")

val hola = Array("Hola", "mundo")
echo(hola: _*)

Hola
Hola
mundo
Hola
mundo


echo: (args: String*)Unit
hola: Array[String] = Array(Hola, mundo)


<a name="subsubseccion-Higher-order functions"></a>
### Higher-order functions

In functional programming, 'higher order functions' are those that return or take as parameters other functions. Among their advantages we must highlight their ability to create abstractions that reduce the duplicity of code and simplify it.

Let's assume, for example, that we need to analyse a file and search the lines that satisfy some criteria with respect to a string (starts or ends with it, for example). We could create a number of functions very similar except for the condition to be checked, or implement a common schema where the condition will be a parameter itself, as we can see in the example below.

In [6]:
import scala.io.Source
val lines = Source.fromFile("files/cadenas.txt").getLines.toList

def buscaEmpieza(cadena: String) =
    for (line <- lines if line.startsWith(cadena)) yield line

def buscaTermina(cadena: String) =
    for (line <- lines if line.endsWith(cadena)) yield line

def buscaContiene(cadena: String) =
    for (line <- lines if line.contains(cadena)) yield line

println("FUNCTIONS:")
buscaEmpieza("hola").foreach(println)
buscaTermina("hola").foreach(println)
buscaContiene("hola").foreach(println)

def buscaCriterio(cadena: String, criterio: (String, String) => Boolean) =
    for (line <- lines if criterio(line, cadena)) yield line
def buscaEmpieza2(cadena: String) = buscaCriterio(cadena, _.startsWith(_))
def buscaTermina2(cadena: String) = buscaCriterio(cadena, _.endsWith(_))
def buscaContiene2(cadena: String) = buscaCriterio(cadena, _.contains(_))

println("\nHIGER ORDER FUNCTIONS:")
buscaEmpieza2("hola").foreach(println)
buscaTermina2("hola").foreach(println)
buscaContiene2("hola").foreach(println)

FUNCTIONS:
hola, qué tal
Al ver a un compañero hay que decir hola
hola, qué tal
Al ver a un compañero hay que decir hola
Adiós don Pepito, hola don José

HIGER ORDER FUNCTIONS:
hola, qué tal
Al ver a un compañero hay que decir hola
hola, qué tal
Al ver a un compañero hay que decir hola
Adiós don Pepito, hola don José


import scala.io.Source
lines: List[String] = List(﻿Hola mundo, hola, qué tal, Al ver a un compañero hay que decir hola, Hola don Pepito, Adiós don José, Adiós don Pepito, hola don José)
buscaEmpieza: (cadena: String)List[String]
buscaTermina: (cadena: String)List[String]
buscaContiene: (cadena: String)List[String]
buscaCriterio: (cadena: String, criterio: (String, String) => Boolean)List[String]
buscaEmpieza2: (cadena: String)List[String]
buscaTermina2: (cadena: String)List[String]
buscaContiene2: (cadena: String)List[String]


<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/optimizar.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important;text-align:left;">
            <ul>
                <li>We could go further by using closures to avoid the first parameter of function `buscaCriterio`</li>
            </ul>
        </td>
    </tr>
</table>

Other typical case that illustrate the benefits of 'higher-order functions' is based on the special methods included in Scala's collections. Let us see a couple of them in the next example.

In [7]:
// Standard version to check whether a list contains negative elements
def containsNegImp(nums: List[Int]): Boolean = {
    var exists = false
    for (num <- nums)
        if (num < 0)
            exists = true
    exists
}

// Simplifying code through higher-order function 'exists'
def containsNegFun(nums: List[Int]): Boolean = nums.exists(_ < 0)

// Both are equivalent
containsNegImp(Nil)
containsNegFun(Nil)
containsNegImp(List(0, -1))
containsNegFun(List(0, -1))

// How can we implement a function to check whether a list contains odd numbers?

<a name="subseccion-Currying"></a>
## Currying

Currying is a functional programming technique that allows us to define a function with multiple lists of parameters instead of just one.

Actually, when we define a curried function with two lists of parameters, we are defining two functions: the first one takes the first list of parameters and returns another function that takes the second list of parameters and returns the final result.

In [8]:
def sumaTradicional(x: Int, y: Int) = x + y
sumaTradicional(1, 2)

def sumaCurried(x: Int)(y: Int) = x + y
sumaCurried(1)(2)

// We can emulate the 'currying' and distinguish the two functions
def sumaSimulandoCurry1(x: Int) = (y: Int) => x + y
sumaSimulandoCurry1(1)(2)

def sumaSimulandoCurry2 = sumaSimulandoCurry1(1)
sumaSimulandoCurry2(2)

sumaTradicional: (x: Int, y: Int)Int
sumaCurried: (x: Int)(y: Int)Int
sumaSimulandoCurry1: (x: Int)Int => Int
sumaSimulandoCurry2: Int => Int
res6: Int = 3


Once we have defined curried function `sumaCurried` and the emulation of currying through functions`sumaSimulandoCurry1` and `sumaSimulandoCurry2`, could we have access to the second implicit function in `sumaCurried`? The answer lies in the partially defined functions previously studied, that is using `_` as follows:

In [9]:
val sumaUno = sumaCurried(1) _
sumaUno(2)

// I we need a function that sums 10 to the argument
val sumaDiez = sumaCurried(10) _
sumaDiez(2)

sumaUno: Int => Int = <function1>
sumaDiez: Int => Int = <function1>
res7: Int = 12


<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/optimizar.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important;text-align:left;">
            <ul>
                <li>We can avoid the blank space before `_` in the previous example because the result is not a valid identifier for Scala. Otherwise, `println_` would be a valid one, so the blank space couldn't be deleted</li>
            </ul>
        </td>
    </tr>
</table>

Currying can be useful when we want to create our own control structures, with a language-alike syntax.

For example, let's assume that we need to create a generic function that performs a given operation on a given file and then closes the file in a secure way. We could implement a higher-order function with two parameters: the file and the operation to be executed. Applying currying, we can separate both parameters into two different lists of one parameter each, and then use another Scala feature (studied before) consisting on substituting the parenthesis by braces when calling a function with just one parameter. so we obtain a syntax similar to a Scala's control structure as we can see in the example below.

In [11]:
import java.io.PrintWriter
import java.io.File

val file = new File("files/date.txt")

// Initial version (without currying)
def withPrintWriter(file: File, op: PrintWriter => Unit) {
    val writer = new PrintWriter(file)
    try {
        op(writer)
    } finally {
        writer.close()
    }
}
withPrintWriter(file,
    writer => writer.println(new java.util.Date)   // _.println(new java.util.Date)
)

// Version with currying
def withPrintWriterCurried(file: File)(op: PrintWriter => Unit) {
    val writer = new PrintWriter(file)
    try {
        op(writer)
    } finally {
        writer.close()
    }
}
withPrintWriterCurried(file){                      // Currying + braces = control structure syntax alike
    writer => writer.println(new java.util.Date)
}


import java.io.PrintWriter
import java.io.File
file: java.io.File = files/date.txt
withPrintWriter: (file: java.io.File, op: java.io.PrintWriter => Unit)Unit
withPrintWriterCurried: (file: java.io.File)(op: java.io.PrintWriter => Unit)Unit


<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/notepad.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important;text-align:left;">
            <ul>
                <li>When we define a 'curried' function we must put the lists of parameters with functions at the end and the rest of parameters at the beginning, so the compiler can infer the types</li>
            </ul>
        </td>
    </tr>
</table>

<a name="subseccion-Parameters by-name and lazy vals"></a>
## Parameters 'by-name' and 'lazy' vals

When a function is called, its parameters are evaluated and then the code of the body of the function is executed with the parameter replaced by their computed values. Nevertheless, sometimes it could be more convenient to evaluate some parameters only when their values are needed.

Let us see it with an example: supose that we want to emulate the behavior of function `assert` in Scala, that throws an `AssertException` when the predicate passed as parameter is evaluated to `false`. Supose that there is a variable `assertionsEnabled` that enables or disables the use of the function. We could implement this functionality in three different ways:

1. We can implement a function with a boolean parameter, as shown in the first version in the example below. In this case the predicate will always be evaluated first, and then the body of the function will be executed.

2. We could use a function literal without params, returning a boolean, as the predicate to be evaluated. In this way, the predicate is evaluated only when its value its needed.

3. Finally, we could use the parameters 'by-name' syntax, `=> type`, in order to define a parameter used in the same way as in version 1, but evaluated like version 2. 

Having in mind the difference between first and third versions of the function, what's the difference in the execution if we introduce as predicate `2 / 0 > 3` and `assertionsEnabled = false`?

In [ ]:
var assertionsEnabled = true

// Primera versión
def myAssert1(pred: Boolean) = {
    if (assertionsEnabled && !pred)
        throw new AssertionError
}
//myAssert1(2 > 3)

// Segunda versión
def myAssert2(pred: () => Boolean) = {
    if (assertionsEnabled && !pred())
        throw new AssertionError
}
//myAssert2(() => 2 > 3)

// Tercera versión
def myAssert3(pred: => Boolean) = {
    if (assertionsEnabled && !pred)
        throw new AssertionError
}
//myAssert3(2 > 3)

Similarly, when we define a variable with `val`, there may be some cases where we want the variable to be evaluated just when it is needed. In this case, Scala provides the `lazy` values, as shown in the next example.

In [1]:
object Demo {
    lazy val x = { println("initializing x"); "done" }  // If we delete 'lazy' it will be evaluated when the object is created
}
Demo

Intitializing Scala interpreter ...

Spark Web UI available at http://172.17.0.2:4040
SparkContext available as 'sc' (version = 2.2.0, master = local[*], app id = local-1515515814515)
SparkSession available as 'spark'


defined object Demo
res0: Demo.type = Demo$@6c2c6999


<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/notepad.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important;text-align:left;">
            <ul>
                <li> The same result can be achieved using `def` (without parameters) instead of `lazy val`. The difference is that the later is evaluated just once and the result is stored inside the variable, while using `def` the expression is evaluated anytime it is needed.</li>
            </ul>
        </td>
    </tr>
</table>

<a name="subseccion-Tail recursion"></a>
## Tail recursion

An iterative function can be defined usually with a loop in an imperative language or with a recursive function in a functional language. Although, in general, the later is shorter and simpler, it is usually branded as less efficient because it stacks the recursive calls in running time. Nevertheless, Scala compiler can perform some optimizations to this recursive schema if it is a tail recursion (if the last sentence in the recursive function is the recursive call to itself). In this way we can get the best of both approaches,

In the following example we can see two versions of a recursive function that throws an exception at the end of a countdown. It is implemented with and without tail recursion. The message printed out by the exception gives a clue about the differences between both versions. In the tail recursive function it appears to be just one recursive call, actually the compiler has optimized the code in order to use the same space in the execution stack.

In [5]:
def boom(x: Int): Int =                       // This IS NOT a tail recursive function
    if (x == 0) throw new Exception("boom!")
    else boom(x - 1) + 1

boom(3)

java.lang.Exception:  boom!

In [4]:
def bang(x: Int): Int =                       // This IS a tail recursive function
    if (x == 0) throw new Exception("bang!")
    else bang(x - 1)

bang(3)

java.lang.Exception:  bang!

<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/notepad.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important;text-align:left;">
            <ul>
                <li>Scala only optimizes tail recursive calls when the called function is the same function that makes the call</li>
            </ul>
        </td>
    </tr>
</table>

<a name="subseccion-Companion `object`"></a>
## Companion `object`

If there is an `object` with the same name of a `class`, this 'singleton' object is attached to the class in a particular way, forming both of them a 'couple' ('companion object' and 'companion class', respectively). Both (the class and the object) must be defined in the same source file and have access to the private members of each other.

For example, apart from class `List`, there is also a `List` object, offering some interesting methods, like:
* `List.apply` creates a List with `List.apply(1, 2, 3)` and, hence, with `List(1, 2, 3)`.
* `List.range(i, j)` returns a list of numbers from `i` to `j-1` (we could use a third parameter to set the increment).
* `List.fill(n)(elem)` returns a list with `elem` repeated `n` times.
* `List.concat(lista1, ..., listaN)` returns a list form by the concatenation of the elements in all the lists passed as parameters.

We could do the same with our types, for example with a functional implementation of Queue, like the example below. Since the functions applied to the beginning of a list are more efficient than those applied to the end of the list, an efficient implementation of Queue can be achieved by representing the queue using two lists: one for the beginning of the queue and the other for the end of the queue reversed.

In [3]:
// 'private' before the list of parameters hides the constructor of the class 
// (the user won't know that two lists are used in this implementation)
class MyQueue[T] private (private val leading: List[T], private val trailing: List[T]) {
    // This method builds the queue, only when required
    private def mirror =
        if (leading.isEmpty) new MyQueue(trailing.reverse, Nil)
        else this

    def head = mirror.leading.head
    def tail = {
        val q = mirror
        new MyQueue(q.leading.tail, q.trailing)
    }
    def append(x: T) = new MyQueue(leading, x :: trailing)
    override def toString = (leading ::: trailing.reverse).toString
}

object MyQueue {
    // constructs a queue with the elements contained in it
    def apply[T](xs: T*) = new MyQueue[T](xs.toList, Nil)
}

val myFunctionalQueue = MyQueue(1, 2, 3)
myFunctionalQueue.head
myFunctionalQueue.tail
myFunctionalQueue.append(4)

defined class MyQueue
defined object MyQueue
myFunctionalQueue: MyQueue[Int] = List(1, 2, 3)
res1: MyQueue[Int] = List(1, 2, 3, 4)


<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/notepad.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important;text-align:left;">
            <ul>
                <li>We could define other constructors for the class, given that the default one has been hidden:</li>
                <ul>
                    <li>`def this() = this(Nil, Nil)`</li>
                    <li>`def this(elems: T*) = this(elems.toList, Nil)`</li>
                </ul>
            </ul>
        </td>
    </tr>
</table>

Let's see another example using a 'companion object' to store a cache in order to save time (and maybe waste space) if the class has a method with a complex an expensive computation. In this case we defined the class `ChecksumAccumulator` that stores the sum of a sequence of bytes in order to compute the checksum for that sequence through the function `checksum`. Having just the class, if we need the checksum of a given string many times, we should create many instances of the class and call the method `checksum` in each instance, with the consequent waste of time. With a 'companion object', we store a map formed by the pairs 'string -> checksum' already processed, providing a method `calculate` that checks whether the checksum has been computed previously, so we can avoid the creation of a new instance of the class and to call method `checksum` if it is not necessary.

In [8]:
class ChecksumAccumulator {
    private var sum = 0 
    def add(b: Byte) { sum += b } 
    def checksum(): Int = ~(sum & 0xFF) + 1
}

import scala.collection.mutable

object ChecksumAccumulator {
    private val cache = mutable.Map[String, Int]()
    
    def calculate(s: String): Int = {
        if (cache.contains(s)){
            cache(s)
        } else {
            val acc = new ChecksumAccumulator
            for (c <- s)
                acc.add(c.toByte)
            val cs = acc.checksum()
            cache += (s -> cs)
            cs
        }
    }
}

ChecksumAccumulator.calculate("Hola mundo!")  // The checksum for the strng is computed and returned
ChecksumAccumulator.calculate("Hola mundo!")  // The checksum stored in the cache is returned

defined class ChecksumAccumulator
import scala.collection.mutable
defined object ChecksumAccumulator
res7: Int = -232


<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/optimizar.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important;text-align:left;">
            <ul>
                <li>This example creates a very simple cache, we usually use a `scala.collection.jcl.WeakHashMap` that automatically frees memory when needed</li>
            </ul>
        </td>
    </tr>
</table>

<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/notepad.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important;text-align:left;">
            <ul>
                <li>An `object` '`X`' does NOT define a type by itself. So we cannot create a variableof type '`X`' (the type is defined by its 'companion class')</li>
            </ul>
        </td>
    </tr>
</table>

<a name="subseccion-Covariant and contravariant types"></a>
## Covariant and contravariant types


Type `List` is covariant, which means that given a type `S`, subtype of `T`, then `List[S]` is a subtype of `List[T]`. Examples:

* `List[String]` is a subtype of `List[Object]`
* `List()` has type `List[Nothing]` and, hence, is a subtype of `List[T]`, for any `T`. This is the erason why `val xs: list[String] = List()` is correct.

Types built using the implementation of `MyQueue` (seen previously) are **not** covariant, so we cannot use a `MyQueue[String]` where a `MyQueue[AnyRef]` is expected.

In [8]:
val q1 = MyQueue("Hola", "Mundo")
var q2: MyQueue[AnyRef] = q1

<console>: 28: error: reference to MyQueue is ambiguous;

We can transform our class `MyQueue` into a covariant type using `+` next to the generic type, as we can see below, but the compiler will return an error, stating that a covariant type appears where a contravariant type is expected. Let us think it carefully: if we define our class as covariant, we could do something like:

<table align="middle" style="border-collapse: collapse; border: none !important; width: 60%;">
    <tr style="border:none !important;">
        <td style="border:none !important; text-align:left">
&nbsp;&nbsp;&nbsp;&nbsp;val q1 = MyQueue("abc")<br>
&nbsp;&nbsp;&nbsp;&nbsp;val q2: MyQueue[AnyRef] = q1<br>
&nbsp;&nbsp;&nbsp;&nbsp;q2.append(List(1, 2, 3))<br>
        </td>
    </tr>
</table>


In [5]:
class MyQueue[+T] private (private val leading: List[T], private val trailing: List[T]) {
    // This method builds the queue, only when required
    private def mirror =
        if (leading.isEmpty) new MyQueue(trailing.reverse, Nil)
        else this

    def head = mirror.leading.head
    def tail = {
        val q = mirror
        new MyQueue(q.leading.tail, q.trailing)
    }
    def append(x: T) = new MyQueue(leading, x :: trailing)
    override def toString = (leading ::: trailing.reverse).toString
}

object MyQueue {
    // constructs a queue with the elements contained in it
    def apply[T](xs: T*) = new MyQueue[T](xs.toList, Nil)
}


<console>: 24: error: covariant type T occurs in contravariant position in type T of value x

We can solve this problem by assigning a type to method `append`. This type must be a supertype of T, which means that we are establishing a 'lower bound' of the type, using `U >: T`.

In [9]:
class MyQueue[+T] private (private val leading: List[T], private val trailing: List[T]) {
    // This method builds the queue, only when required
    private def mirror =
        if (leading.isEmpty) new MyQueue(trailing.reverse, Nil)
        else this

    def head = mirror.leading.head
    def tail = {
        val q = mirror
        new MyQueue(q.leading.tail, q.trailing)
    }
    def append[U >: T](x: U) = new MyQueue[U](leading, x :: trailing)
    override def toString = (leading ::: trailing.reverse).toString
}

object MyQueue {
    // constructs a queue with the elements contained in it
    def apply[T](xs: T*) = new MyQueue[T](xs.toList, Nil)
}

val q1 = MyQueue("Hola", "Mundo")
var q2: MyQueue[AnyRef] = q1

defined class MyQueue
defined object MyQueue
q1: MyQueue[String] = List(Hola, Mundo)
q2: MyQueue[AnyRef] = List(Hola, Mundo)


<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/notepad.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important;text-align:left;">
            <ul>
                <li>Scala also offers the functionality for `[-T]` instead of `[+T]` or `[U <: T]` instead of `[U >: T]`, but the details of these notations are out of the scope of this notebook.</li>
            </ul>
        </td>
    </tr>
</table>

<a name="subseccion-Implicit conversions"></a>
## Implicit conversions


Previously on this notebook we studied an implementation of rational numbers and their operators, such as `+` to perform the sum of rational numbers. It should be easy now to override this operator in order to sum an integer to a rational. 

The idea is to enable operations like: (1/2 + 3/4) and (1/2 + 3), but what will happen if we want to reverse the operands? (3 + 1/2)

In [17]:
class Rational6(val n: Int, val d: Int){
    require(d != 0)
    def this(n: Int) = this(n, 1)
    override def toString = n+"/"+d
    
    def +(otro: Rational6): Rational6 = 
        new Rational6(
            n * otro.d + otro.n * d,
            d * otro.d
        )
    
    def +(otro: Int): Rational6 = 
        new Rational6(
            n + otro * d,
            d
    )
}
new Rational6(1, 2) + 3
3 + new Rational6(1, 2)

defined class Rational6


The error is caused because Int type do not have an operator that receives a rational number as parameter. This is a typical case to introduce implicit conversions from `Int` to `Rational`, in such way that the sum could be carried out.

In [18]:
implicit def intToRational(x: Int) = new Rational6(x)
3 + new Rational6(1, 2)

intToRational: (x: Int)Rational6
res10: Rational6 = 7/2


<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/warning.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important; text-align:left">
            <ul>
                <li>To apply the implicit conversion, it must be accessible from the code where it is needed</li>
            </ul>
        </td>
    </tr>
</table>

There are 'enriched' versions of basic types (`Byte`, `Short`, `Int`, `Char`, `String`, `Float`, `Double`, `Boolean`) that allow us to perform many more operations with them. For example, with the implicit conversion from `String` to `RichString` we can apply sequence operators on a String (taken as a sequence of characters). 

In [19]:
def tieneMay(s: String) = s.exists(_.isUpper)
tieneMay("hola mundo")
tieneMay("Hola Mundo")

tieneMay: (s: String)Boolean
res11: Boolean = true


In general, we must observe a set of rules in order to correctly define and apply implicit conversions:

* Only those conversions tagged as `implicit` are applied, because it would be a mess if we could do the conversion using any method.
* The conversion must be located into the scope and use a unique identificator, so Scala libraries usually have an object called `Preamble` that contains the conversions and use them easily by importing it, `import Preamble._`. There is an exception to this rule: the compiler searches the conversion also into the companion objects of the expected types. For example, we can define an implicit conversion `dollarToEuro` inside the companion object `Dollar`.
* An implicit conversion is applied **only** if it is the only available one. If there are more than one available conversions, the compiler should choose one of them and it would be hard to debug possible errors.
* Compiler applies conversions one by one, it does **not** solve nested conversions.
* If the code works as it is, the compiler does **not** try to apply any conversions.
* The name given to a conversion is useful just to import and use them (for example, if we want to import some conversions and no others)
* Conversions are applied in three cases:
  * Implicit conversion to an expected type. I.e.: `val x:Int = 3.5` if conversion `intToDouble` is defined
  * Conversions of the receiver of a selection. I.e.: `"abc".exists`
  * Implicit parameters. I.e.: `def greet(name: String)(implicit prompt: PreferredPrompt)` if an `implicit val` is defined with type `PreferredPrompt`

<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/notepad.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important;text-align:left;">
            <ul>
                <li> Implicit conversions allow us to extend the functionality of third party libraries, that we have to use them as they are without modifying them.</li>
            </ul>
        </td>
    </tr>
</table>

<a name="subseccion-Pattern matching and `case` classes"></a>
## Pattern matching and `case` classes


Pattern matching is a key tool in Scala for the processing of recursive data structures (specially tree structures), among other things. Pattern matching can be seen as a generalization of 'switch' clause of other programming languages, in such way that it checks an expression against a number of patterns and decides which way to take next, depending on the pattern matched. But pattern matching in Scala is much more powerful than that, because it includes new pattern types.

In the case of lists, for example, we can create functions that decides what must be done depending on the structure of the list, checking whether it is an empty list, or it contains just one element, or if the first element is equal to '1', etc. In the example below, we can see the use of `match` clause to set the expression to be matched with the patterns, which are preceded by `case`. Finally, we use the arrow `=>` to separate the pattern from the expressions to be executed in case of a matching. Pattern `_` serves as wild-card, and symbol `*` indicates a sequence of elements (possibly empty). Anyway, we will review more pattern types later.

The order of the patterns is relevant, as we can see in the example, calling `analizaLista(List(1))`, because it matches two patterns. For this reason, we must put first the patterns from the most concrete to the most generic ones.

In [8]:
def analizaLista(lista: List[Int]): String = {
    lista match {
        case Nil => "It's empty"
        case List(e) => "It has just one element ("+e+")"    // It could be also written like 'case e :: Nil =>'
        case List(1, _*) => "Starts by one"                  // It could be also written like'case 1 :: resto =>'
        case _ => "Ohter case"
    }
}

println(analizaLista(Nil))
println(analizaLista(List(1)))
println(analizaLista(List(1, 2, 3)))
println(analizaLista(List(2, 3)))

It's empty
It has just one element (1)
Starts by one
Ohter case


analizaLista: (lista: List[Int])String


<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/notepad.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important;text-align:left;">
            <ul>
                <li>The patterns for `List` in the example work because an 'extractor' has been previously defined (we will study them later)</li>
                <li>The pattern `x :: xs` works because, apart from being an operator of `List`, there is in Scala a class `scala.::`, so the pattern is interpreted as `::(x, xs)`</li>
            </ul>
        </td>
    </tr>
</table>

In Scala, we dan define our own `case classes` to be used in a pattern matching structure. A common case is the analysis of arithmetic expressions, that can be variables, binary operations, etc. In the example below, we show the use of `case classes` (one for each type of arithmetic expressions) for pattern matching.

When we define these classes as `case class`, we must take into account that:

* A factory is automatically created, in order to build objects of the class without writting `new`, i.e. `val v = Var("x")` instead of `val v = new Var("x")`,... (just as we did with `List`). Since we can avoid the `new`s, it is easier to define nested expressions, like in `val op = BinOp("+", Number(1), v)`.

* The parameters of the case classes are declared as `val` so they are the fields of the classes.

* The compiler automatically adds simple implementations for the methods `toString`, `hashCode` and `equals`.

In [10]:
abstract class Expr
case class Var(name: String) extends Expr
case class Number(num: Double) extends Expr
case class UnOp(operator: String, arg: Expr) extends Expr
case class BinOp(operator: String, left: Expr, right: Expr) extends Expr

val v = Var("x")                       // 'new' is not necessary
println(v.name)                        // The parameters are defined as 'val'
val op = BinOp("+", Number(1), v)      // Easy to define nested expressions
println(op)                            // There is an implementation for 'toString'...
op.right == Var("x")                   // ... and 'equals'

x
BinOp(+,Number(1.0),Var(x))


defined class Expr
defined class Var
defined class Number
defined class UnOp
defined class BinOp
v: Var = Var(x)
op: BinOp = BinOp(+,Number(1.0),Var(x))
res8: Boolean = true


Let's assume that we need to carry out some simplifications to arithmetics expressions. In this example we see some pattern types:

* constants: like `"+"`, where will match any value equal to that, according to `==`
* variables: like `e` used to match any value, and then use it in the pattern expressions 
* wild card : `_`, matches any value, but it cannot be recovered to use it, it can be nested (i.e. `UnOp("-",_)`)
* constructors: like `UnOp("-",e)`, similar to the constructor of the class, although its parameters are patterns as well

In [12]:
def simplifyTop(expr: Expr): Expr = expr match {
    case UnOp("-", UnOp("-",e)) => e               // doble negation
    case BinOp("+", e, Number(0)) => e             // sum zero
    case BinOp("*", e, Number(1)) => e             // multiply by one
    case _ => expr
}

simplifyTop(UnOp("-", UnOp("-", Var("x"))))
simplifyTop(UnOp("-", UnOp("-", BinOp("+", Number(0), Var("x")))))  // It just simplifies the upmost level

simplifyTop: (expr: Expr)Expr
res10: Expr = BinOp(+,Number(0.0),Var(x))


<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/notepad.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important;text-align:left;">
            <ul>
                <li>Differences with Java's `switch`:
                    <ul><li> `match` is an expression</li>
                        <li> It does **not** need a `break`</li> 
                        <li> If there is no match with some of its patterns, a `MatchError` exception is thrown</li>
                        <li> We can use many types of patterns, not only constants</li>
                    </ul>
                </li>
                <!--
                <li>Si tenemos constantes en nuestro código con el mismo nombre que la variable que usamos en un patrón puede haber confusión sobre si se está cogiendo el valor de la constante o no, y en Scala se resuelve cogiendo la constante si empieza por mayúscula o la escribimos entre comillas simples ``</li>
                -->
            </ul>
        </td>
    </tr>
</table>

There are more types of patterns, such as:
* sequences : `_*` represents a number of elements in a sequence, even zero (i.e.: `List(1, _*)`)
* tuples : with a pattern like `(a, b, c)` we can match any tuple with 3 elements
* types : a pattern like `case s: String =>` or `case m: Map[_,_] =>` we can match the type of the expression
* assignment : with `<variable> @ <patrón>` we can match the pattern and then assign the variable to retrieve the value 

In [14]:
def analizandoPatrones(x: Any) = x match {
    case List(1, _*) => "It's a list starting by 1"
    case (a, b, c) => "It's a tuple with three elements"
    case s: String => "It's a String (with a length of "+s.length+")" // Note that x.length is not correct, cause it's type Any
    case UnOp("abs", e @ UnOp("-", _)) => "It's an application of 'abs' on a negation: "+e
    case _ => "I don't know what it is!"
}
println(analizandoPatrones(List(1)))
println(analizandoPatrones("hola"))
println(analizandoPatrones(("hola", 10, List(1))))
println(analizandoPatrones(UnOp("abs", UnOp("-", Number(1)))))

It's a list starting by 1
It's a String (with a length of 4)
It's a tuple with three elements
It's an application of 'abs' on a negation: UnOp(-,Number(1.0))


analizandoPatrones: (x: Any)String


<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/notepad.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important;text-align:left;">
            <ul>
                <li>It is not possible to determine through a pattern if, for example, an expression is a amp from Int to Int (`case m: Map[Int, Int]`). Scala can just see if m is a map.</li>
            </ul>
        </td>
    </tr>
</table>

Sometimes a pattern is not enough to express a complex condition, it can be overcome by using `if <condition>` between the pattern and the arrow `=>`.

In [20]:
def analizandoPatronesCondicionados(x: Any) = x match {
    case n: Int if 0 < n => "It is positive"
    case s: String if s(0) == 'a' => "It starts by 'a'" 
    case _ =>
}

analizandoPatronesCondicionados: (x: Any)Any


In the previous example, pattern `case _` is necessary in order to tackle all the cases. Remember that a `MatchError` exception is thrown whenever an expression does not match any pattern. We can write, for example, `case _ =>` if we do not want to treat those cases, so `match` will return a type `Unit`.

Supose that we do not use `case _` pattern, because we are covering all possible cases of arithmetic expressions in our pattern-matching structure. In that case, someone could define a new type of arithmetic expression in a different module, in such way that our pattern-matching structure will cause errors. To overcome this problem, Scala allows us to "seal" those classes that are supertypes of our case classes, in order to tell the compiler to prevent the creation of new case classes outside this file.

Regarding our example with arithmetic expressions, we can seal the class `Expr` by preceding it with `sealed` (`sealed abstract class Expr`). In addition, since the compiler already knows all the possible cases for an object of type  `Expr`, it could warn us when we create a match with uncovered cases.

In [21]:
sealed abstract class Expr
case class Var(name: String) extends Expr
case class Number(num: Double) extends Expr
case class UnOp(operator: String, arg: Expr) extends Expr
case class BinOp(operator: String, left: Expr, right: Expr) extends Expr

def describe(e: Expr): String = e match {  // it will arise a compiler warning cause we aren't covering all possible cases
    case Number(_) => "a number"
    case Var(_) => "a variable"
}

<console>:23: warning: match may not be exhaustive.
It would fail on the following inputs: BinOp(_, _, _), UnOp(_, _)
       def describe(e: Expr): String = e match {  // it will arise a compiler warning cause we aren't covering all possible cases
                                       ^
defined class Expr
defined class Var
defined class Number
defined class UnOp
defined class BinOp
describe: (e: Expr)String


<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/notepad.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important;text-align:left;">
            <ul>
                <li>If the compiler warns us because we have a sealed class and a match without covering all the cases, but we are sure that the code is correct, we could avoid the warning by writting `(e: @unchecked)` instead of just `e` before the `match` in the example</li>
            </ul>
        </td>
    </tr>
</table>

Another utility of pattern matching is in the assignment of variables.

In [22]:
// Assigning variables with patterns
val myTuple = (123, "abc")
val (n, s) = myTuple

val miExp = new BinOp("*", Number(5), Number(1)) // It works without 'new' as well
val BinOp(op, left, right) = miExp

myTuple: (Int, String) = (123,abc)
n: Int = 123
s: String = abc
miExp: BinOp = BinOp(*,Number(5.0),Number(1.0))
op: String = *
left: Expr = Number(5.0)
right: Expr = Number(1.0)


And within the generators in `for` loops, discharging those elements that do not match the pattern.

In [23]:
val results = List(UnOp("-", Number(1)), Number(2), BinOp("+", Number(1), Var("x")))
for (UnOp("-",e) <- results) 
    println(e)

Number(1.0)


results: List[Product with Serializable with Expr] = List(UnOp(-,Number(1.0)), Number(2.0), BinOp(+,Number(1.0),Var(x)))


Finally, we can use patterns (through a sequence of `case`s in braces) wherever a function literal is expected, in order to create different entry points depending on the patterns specified.

In [26]:
val miFuncion: Any => Unit = {
    case s:String => println("Str:"+s)
    case x:Int if x >= 0 => println("Pos:"+x)
    case x:Int if x < 0 => println("Neg:"+x)
}

println(miFuncion("2"))
println(miFuncion(2))
println(miFuncion(-2))

Str:2
()
Pos:2
()
Neg:-2
()


miFuncion: Any => Unit = <function1>


<a name="subseccion-Type `Option`"></a>
## Type `Option`

In Scala we can decide that some value is optional through type `Option`, that can contain a value `x` of any type with the value `Some(x)` or it can have a value `None`.

An example: the method `get` of type `Map` returns a `Some(value)` if the specified key is contained in the map, or `None` in the other case. Method `find` of type `List` works in the same way.

On the other hand, the most common mechanism to extract the real value of an `Option` is by using pattern matching, as we can see in the next example.

In [27]:
val capitals = Map("France" -> "Paris", "Japan" -> "Tokyo")

def show(x: Option[String]) = x match {
    case Some(s) => s
    case None => "?"
}
show(capitals get "France")
show(capitals get "North Pole")

val provincias = List("Jaén", "Córdoba", "Sevilla", "Huelva", "Cádiz", "Málaga", "Granada", "Almería")
provincias.find(p => p.endsWith("a"))     // Returns the first element that satisfies the condition, or None

capitals: scala.collection.immutable.Map[String,String] = Map(France -> Paris, Japan -> Tokyo)
show: (x: Option[String])String
provincias: List[String] = List(Jaén, Córdoba, Sevilla, Huelva, Cádiz, Málaga, Granada, Almería)
res16: Option[String] = Some(Córdoba)


<a name="subseccion-Extractors"></a>
## Extractors


Now that we can use pattern matching, let us see how can we create our own patterns. In the previous section we have used `Some(x)` as a pattern, because `Some` is a `case class`. But, what if we need to analyse a String in order to check whether it is an Email or not, and looking for the username and the extension? Take into account that `String` is not a case class and, if we need to extract the username and/or extension of the email, we need a method that locates the `@` symbol inside the string and returns the required parts. The best solution would use a pattern like `Email(user, domain)`, which will also serve to aditional purposes, such as manage lists of emails, etc.

The tool intended to allow us to create new patterns for existing types, not necessarily based on the internal structure of the type, is the `extractor`. Basically, it consists in:
* A method `unapply`: to match and separate the values
* A method `apply` (**optional**): to build the values 

In [2]:
object EMail {
    def unapply(str: String): Option[(String, String)] = {           // Extraction (mandatory)
        val parts = str split "@"
        if (parts.length == 2) Some(parts(0), parts(1)) else None
    }

    def apply(user: String, domain: String) = user +"@"+ domain      // Injection (optional)
}

object UpperCase {                                                   // Example without 'apply'
    def unapply(s: String): Boolean = s.toUpperCase == s
}

"HOLA@myDomain.com" match {
    case EMail(user @ UpperCase(), domain) => println("USER: '"+user+"' DOMAIN: '"+domain+"'")
    case EMail(user, domain) => println("user: '"+user+"' domain: '"+domain+"'")
    case _ => 
}

val email: String = EMail("john_doe", "myDomain.com")

USER: 'HOLA' DOMAIN: 'myDomain.com'


defined object EMail
defined object UpperCase
email: String = john_doe@myDomain.com


It is also possible to create patterns with an undetermined number of elements. To taht end, we need to replace method `unapply` by method `unapplySeq`. We see an example below.

In [3]:
object Domain {
    def unapplySeq(whole: String): Option[Seq[String]] =
        Some(whole.split("\\.").reverse)

    def apply(parts: String*): String =
        parts.reverse.mkString(".")
}

def procesarDominio(email: String) = email match {
    case EMail(_, Domain("es", "us", _*)) => println("Dominio 'us.es'")
    case EMail(_, Domain("es", _*)) => println("Dominio 'es'")
    case _ => 
}

procesarDominio("john@us.es")

Dominio 'us.es'


defined object Domain
procesarDominio: (email: String)Unit


<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/notepad.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important;text-align:left;">
            <ul>
                <li>Remember that the order of the patterns is relevant: we must put first the most restrictive ones.</li>
                <li>Using `unapplySeq` we could also return a variable part, such as the user and the sequence of domains.</li>
            </ul>
        </td>
    </tr>
</table>

<table align="left" style="border-collapse: collapse; border: none !important; width: 100%;">
    <tr style="border:none !important;">
        <td style="border:none !important; width: 60px;">
<img src="icons/optimizar.png" align="left" width="50px"> 
        </td>
        <td style="border:none !important; text-align:left">
            <ul>
                <li>El uso de `case class` es en principio más eficiente que el uso de un extractor, aunque los extractores ofrecen mayor independencia de la representación interna de las clases, pero en cualquier caso los patrones funcionarán exactamente igual</li>
            </ul>
        </td>
    </tr>
</table>

<a name="subsubseccion-Regular expressions "></a>
### Regular expressions 


Regular expressions constitute another really useful tool for working with patterns. They have the same syntax and use as in Java. First, the regular expression is defined through a `String`, specifying with parentheses the groups corresponding to the parts of the string to be matched. For example, we can match a decimal number and its three components (sign, integer part, and fractional part) with the regular expression `(-)?(\d+)(\.\d*)?`, where the parentheses define the three parts.

There are also predefined methods to work with the pattern, like:
* `findFirstIn`
* `findAllIn`
* `findPrefixOf`

Finally, every regular expression in Scala defines an extractor, so we can use them like such.

In [4]:
import scala.util.matching.Regex

val Decimal = new Regex("(-)?(\\d+)(\\.\\d*)?")       // Regex for decimal numbers
//val Decimal = new Regex("""(-)?(\d+)(\.\d*)?""")    // Equivalent, using raw strings
//val Decimal = """(-)?(\d+)(\.\d*)?""".r             // Equivalent, using method 'r' from RichString

for (s <- Decimal findAllIn "for -1.0 to 99 by 3") println(s)

val Decimal(sign, integerPart, decimalPart) = "-1.24" // What happens if we omit the symbol?

"-1.24" match {
    case Decimal("-", _, _) => println("negative")
    case _ => println("positive")
}

-1.0
99
3
negative


import scala.util.matching.Regex
Decimal: scala.util.matching.Regex = (-)?(\d+)(\.\d*)?
sign: String = -
integerPart: String = 1
decimalPart: String = .24


---